In [12]:
import os
import json
from pprint import pprint

# List to hold the data from all JSON files
all_json_data = {}

# Get the current directory
current_directory = os.getcwd()

# Loop through all files in the current directory
for filename in os.listdir(current_directory):
    if filename.endswith('.json'):
        file_path = os.path.join(current_directory, filename)
        
        # Open and read the JSON file
        with open(file_path, 'r') as file:
            try:
                json_data = json.load(file)

                label = filename.split('.')[0]

                all_json_data[label] = json_data
            except json.JSONDecodeError as e:
                print(f"Error reading {filename}: {e}")

# Now, all_json_data contains the contents of all JSON files
pprint(all_json_data.keys())


dict_keys(['openai_drag_success_cases_image_few_shot', 'openai_drag_failure_cases_image_few_shot', 'openai_rabi_success_cases_image_few_shot', 'openai_gmm_failure_cases_image_few_shot', 'openai_gmm_success_cases_image_few_shot', 'openai_rabi_failure_cases_image_few_shot', 'openai_resonator_spec_success_cases_raw', 'openai_drag_success_cases_raw', 'openai_gmm_failure_cases_raw', 'openai_rabi_failure_cases_raw', 'openai_resonator_spec_failure_cases_raw', 'openai_resonator_spec_success_cases_image_few_shot', 'openai_drag_failure_cases_raw', 'openai_resonator_spec_failure_cases_image_few_shot', 'openai_rabi_success_cases_raw', 'openai_gmm_success_cases_raw'])


In [13]:
import pandas as pd

In [14]:
entries = []

def summarize_statistics(data):

    entries = []
    for key,val in data.items():
        data = {
            'model_type':key
        }
        data.update(val)
        entries.append(data)

    return entries

for case,data in all_json_data.items():
    model = case.split('_')[0]
    case = '_'.join(case.split('_')[1:-1])
    for _, results in data.items():
        entry = {}
        entry['case'] = case
        entry['model'] = model
        for r in summarize_statistics(results):
                r.update(entry)
                entries.append(r)

In [15]:
print(entries[0].keys())

dict_keys(['model_type', 'full', 'fitting_only', 'visual_only', 'case', 'model'])


In [71]:
df = pd.DataFrame(entries)

In [72]:
df

,model_type,full,fitting_only,visual_only,case,model
0,no_image_few_shot,{'plot': '1. The slope of the blue trend line ...,{'fitting': 'Sweep start: -0.00781943733228437...,{'plot': '1. The slope of the blue trend line ...,drag_success_cases_image_few,openai
1,image_few_shot,{'plot': '1. The slopes of the trend lines are...,{'fitting': 'Sweep start: -0.00781943733228437...,{'plot': '1. The slopes of the trend lines are...,drag_success_cases_image_few,openai
2,no_image_few_shot,{'plot': '1. The slopes of the trend lines are...,{'fitting': 'Sweep start: -0.00872015428886839...,{'plot': '1. The slopes of the trend lines are...,drag_success_cases_image_few,openai
3,image_few_shot,{'plot': '1. The slope of the blue trend line ...,{'fitting': 'Sweep start: -0.00872015428886839...,{'plot': '1. The slope of the blue trend line ...,drag_success_cases_image_few,openai
4,no_image_few_shot,{'plot': '1. The slope of the blue trend line ...,{'fitting': 'Sweep start: -0.00736063098764201...,{'plot': '1. The slope of the blue trend line ...,drag_success_cases_image_few,openai
...,...,...,...,...,...,...
635,image_few_shot,{'plot_hexbin': 'The plot shows three clusters...,"{'fitting': 'Signal to noise ratio: {(0, 1): 1...",{'plot_hexbin': 'The plot shows three clusters...,gmm_success_cases,openai
636,no_image_few_shot,{'plot_hexbin': 'The plot shows three visible ...,"{'fitting': 'Signal to noise ratio: {(0, 1): 1...",{'plot_hexbin': 'The plot shows three visible ...,gmm_success_cases,openai
637,image_few_shot,{'plot_hexbin': 'The plot shows three distinct...,"{'fitting': 'Signal to noise ratio: {(0, 1): 1...",{'plot_hexbin': 'The plot shows three distinct...,gmm_success_cases,openai
638,no_image_few_shot,{'plot_hexbin': 'The plot shows three visible ...,"{'fitting': 'Signal to noise ratio: {(0, 1): 1...",{'plot_hexbin': 'The plot shows three visible ...,gmm_success_cases,openai


In [73]:
mask = df['case'].str.contains('success')
df['success'] = mask

In [74]:
configs = ["full", "fitting_only", "visual_only"]

for config in configs:
    # if the json entry in df[config] has a key "success" then the model was successful
    mask = df[config].apply(lambda x: x.get('Experiment success', False)==True)
    df[config+" success"] = mask
    
for config in configs:
    df[config+" correct"] = df[config+" success"] == df['success']

In [75]:
df

,model_type,full,fitting_only,visual_only,case,model,success,full success,fitting_only success,visual_only success,full correct,fitting_only correct,visual_only correct
0,no_image_few_shot,{'plot': '1. The slope of the blue trend line ...,{'fitting': 'Sweep start: -0.00781943733228437...,{'plot': '1. The slope of the blue trend line ...,drag_success_cases_image_few,openai,True,True,True,True,True,True,True
1,image_few_shot,{'plot': '1. The slopes of the trend lines are...,{'fitting': 'Sweep start: -0.00781943733228437...,{'plot': '1. The slopes of the trend lines are...,drag_success_cases_image_few,openai,True,True,True,True,True,True,True
2,no_image_few_shot,{'plot': '1. The slopes of the trend lines are...,{'fitting': 'Sweep start: -0.00872015428886839...,{'plot': '1. The slopes of the trend lines are...,drag_success_cases_image_few,openai,True,True,True,True,True,True,True
3,image_few_shot,{'plot': '1. The slope of the blue trend line ...,{'fitting': 'Sweep start: -0.00872015428886839...,{'plot': '1. The slope of the blue trend line ...,drag_success_cases_image_few,openai,True,True,True,True,True,True,True
4,no_image_few_shot,{'plot': '1. The slope of the blue trend line ...,{'fitting': 'Sweep start: -0.00736063098764201...,{'plot': '1. The slope of the blue trend line ...,drag_success_cases_image_few,openai,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,image_few_shot,{'plot_hexbin': 'The plot shows three clusters...,"{'fitting': 'Signal to noise ratio: {(0, 1): 1...",{'plot_hexbin': 'The plot shows three clusters...,gmm_success_cases,openai,True,True,True,True,True,True,True
636,no_image_few_shot,{'plot_hexbin': 'The plot shows three visible ...,"{'fitting': 'Signal to noise ratio: {(0, 1): 1...",{'plot_hexbin': 'The plot shows three visible ...,gmm_success_cases,openai,True,True,True,True,True,True,True
637,image_few_shot,{'plot_hexbin': 'The plot shows three distinct...,"{'fitting': 'Signal to noise ratio: {(0, 1): 1...",{'plot_hexbin': 'The plot shows three distinct...,gmm_success_cases,openai,True,False,True,False,False,True,False
638,no_image_few_shot,{'plot_hexbin': 'The plot shows three visible ...,"{'fitting': 'Signal to noise ratio: {(0, 1): 1...",{'plot_hexbin': 'The plot shows three visible ...,gmm_success_cases,openai,True,True,True,True,True,True,True


In [77]:
# get the ratio of True in "full success"
# group by the value of "case"
df.groupby(['case'])['fitting_only success'].mean()

case
drag_failure_cases                        0.400
drag_failure_cases_image_few              0.400
drag_success_cases                        0.950
drag_success_cases_image_few              0.950
gmm_failure_cases                         1.000
gmm_failure_cases_image_few               1.000
gmm_success_cases                         1.000
gmm_success_cases_image_few               1.000
rabi_failure_cases                        1.000
rabi_failure_cases_image_few              1.000
rabi_success_cases                        1.000
rabi_success_cases_image_few              1.000
resonator_spec_failure_cases              0.925
resonator_spec_failure_cases_image_few    0.925
resonator_spec_success_cases              0.975
resonator_spec_success_cases_image_few    0.975
Name: fitting_only success, dtype: float64